In [1]:
import os

from kloppy import sportec
import pandas as pd
from tqdm import tqdm

import pandas as pd
import numpy as np
root_path = "../../../rdf/sp161/shared/soccer-decision-making-r/sportec"
game = "DFL-MAT-J03YGJ.xml"

games = os.listdir(f"{root_path}/tracking/xml")
game = games[8]

In [2]:
event = sportec.load_event(
        event_data= f"{root_path}/event/{game}",
        meta_data=f"{root_path}/match_information/{game}",
        coordinates="sportec",
        event_types=["pass"],
    ).to_df()

event['event_id'] = event['event_id'].astype(int)
event['match_id'] = game
tracking = sportec.load_tracking(
        raw_data= f"{root_path}/tracking/xml/{game}",
        meta_data=f"{root_path}/match_information/{game}",
        sample_rate=1,
        coordinates="sportec",
        only_alive=False,
    ).to_df()

In [3]:
kpi_path = f"{root_path}/KPI_Merged_all/KPI_MGD_{game[:-4]}.csv"
kpi_df = pd.read_csv(kpi_path ,sep = ';', encoding='latin-1', on_bad_lines='skip')


In [ ]:
kpi_df = kpi_df.drop_duplicates(subset = "FRAME_NUMBER").sort_values(by = "FRAME_NUMBER")
kpi_df['NEXT_FRAME'] = kpi_df['FRAME_NUMBER'].shift(-1)
kpi_df['NEXT_FRAME'] = kpi_df['NEXT_FRAME'].fillna(kpi_df['FRAME_NUMBER'].max())
kpi_df = kpi_df[kpi_df['NEXT_FRAME'] - kpi_df['FRAME_NUMBER'] >= 10]

In [37]:
import xml.etree.ElementTree as ET
def build_player_map(path): 
    tree = ET.parse(path) 
    root = tree.getroot() 
    data = {} 
    for root_elem in root.findall('.//Team'): 
        TeamId = root_elem.attrib.get('TeamId') 
        for elem in root_elem.findall('.//Player'): 
            data[elem.attrib.get('PersonId')] = TeamId 
    return data
player_map = build_player_map(f"/home/lz80/rdf/sp161/shared/soccer-decision-making-r/sportec/match_information/{game}")
player_map['BALL'] = 'BALL'
tracking['x_velo'] = tracking['ball_x'] - tracking['ball_x'].shift(-10)
tracking['y_velo'] = tracking['ball_y'] - tracking['ball_y'].shift(-10)

In [40]:
t = pd.merge(kpi_df, event[['event_id', 'result']], left_on = "EVENT_ID", right_on = "event_id")
t = t.sort_values(by = "FRAME_NUMBER")


def get_rec(row, tracking):
    if not pd.isna(row['RECFRM']):
        return row['RECFRM']
    frame = row['FRAME_NUMBER']
    next_frame = tracking['frame_id'].max() if pd.isna(row['NEXT_FRAME']) else row['NEXT_FRAME']
    post_frame_track = tracking[(tracking['frame_id'] >= frame) & ((tracking['frame_id'] <= next_frame))].copy()
    if not pd.isna(row['PUID2']):
        #if we know the receiver, choose frame where receiver is closest
        recept_player = row['PUID2']
        post_frame_track['ball_dist'] = (post_frame_track['ball_x'] - post_frame_track[f'{recept_player}_x'])**2 + (post_frame_track['ball_y'] - post_frame_track[f'{recept_player}_y'])**2
        min_index = post_frame_track['ball_dist'].idxmin()
        print(post_frame_track.loc[min_index, 'frame_id'])
        return post_frame_track.loc[min_index, 'frame_id']
    if row['result'] == 'OUT':
        oob = oob_df[oob_df['frame_id'] >= frame]
        return oob['frame_id'].iloc[0]
    #else: perform 20 degree interception approach
    pass_x, pass_y = post_frame_track[post_frame_track['frame_id'] == frame][['x_velo', 'y_velo']].iloc[0].values

    p = np.array([pass_x, pass_y], dtype=float)
    p_norm = np.linalg.norm(p)

    v = post_frame_track[["x_velo", "y_velo"]].to_numpy(dtype=float)
    v_norm = np.linalg.norm(v, axis=1)
    dot = v @ p 
    cos_theta = dot / (v_norm * p_norm)
    cos_theta = np.clip(cos_theta, -1.0, 1.0)
    post_frame_track["angle_deg"] = np.degrees(np.arccos(cos_theta))

    intercept_mask = (post_frame_track['angle_deg'] >= 20) | ((abs(post_frame_track['ball_x']) > (105/2 - .02))  | (abs(post_frame_track['ball_y']) > (34 - .02)))
    over_twenty = post_frame_track[intercept_mask]

    if over_twenty.shape[0] == 0:
        return min(row['NEXT_FRAME'], frame + 50, tracking['frame_id'].max())
    return over_twenty['frame_id'].iloc[0]

t['n_RECFRM'] = t.apply(lambda x: get_rec(x, tracking), axis = 1)
t= t[(t['n_RECFRM'] - t['FRAME_NUMBER']) >= 10]
#frames = pd.merge(t, blocks, on = 'FRAME_NUMBER')

#frames[['EVENT_ID', 'NEXT_FRAME_SAME_CLUB', 'NEXT_FRAME_DIFF_CLUB', 'FRAME_NUMBER']].sort_values(by = 'FRAME_NUMBER')

10641
10741
10848
13639
14387
16289
17430
17434
18142
18249
18388
18543
18648
21544
22769
26215
26793
28204
30272
30673
30722
31085
31439
31581
31595
34750
34781
37491
39204
40882
40955
41600
43806
43865
44201
44336


<positron-console-cell-40>:30: RuntimeWarning: invalid value encountered in divide
<positron-console-cell-40>:30: RuntimeWarning: invalid value encountered in divide


46144
46318
49237
49669
50133
50176
53349
53471
53561
54752
54948
57441
57798
57887
59300
59606
62106
63390
65108
67680
70818
71231
72936
72972
74112
74293
75408
75469
75644
79176
79678
81216
104869
111021
111192
111588
114261
114502
114600
114804
117715
119570
119616
119749
120064
122236
124582
125880
127258
128583
129670
129791
131980
132637
133046
133303
133779
134272
134602
135049
135098
138345
139860
141262
141333
141356
142104
142707
142773
145167
148267
149765
150084
150462
150491
151483
151945
153524
153612
154223
155597
156017
156441
157255
158239
162082
162428
162652
163320
163461
163540
163655
164382
164633
165268
165754
165997
167679
168141
168967
169105
169200
170774
171886
172374
172760
172888
173107
173630
173806
175228
175520
175863


<positron-console-cell-40>:30: RuntimeWarning: invalid value encountered in divide


In [41]:
def tracking_wide_to_long(df: pd.DataFrame) -> pd.DataFrame:
    """
    Converts wide kloppy output of tracking data to more manageable long format

    Inputs:
        df - wide kloppy output
    Returns
        df in long format
    """
    track_cols = [c for c in df.columns if c.startswith("DFL-OBJ-") and c.count("_") >= 1]

    id_cols = [c for c in df.columns if c not in track_cols]
    m = df.melt(
        id_vars=id_cols,
        value_vars=track_cols,
        var_name="var",
        value_name="value"
    )

    obj_metric = m["var"].str.rsplit("_", n=1, expand=True)
    m["object_id"] = obj_metric[0]
    m["metric"] = obj_metric[1]

    m = m.drop(columns=["var"])

    print(m)
    print([c for c in id_cols if m[c].isna().any()])
    long_df = (
        m.pivot_table(
            index=id_cols + ["object_id"],
            columns="metric",
            values="value",
            aggfunc="first" 
        )
        .reset_index()
    )
    print(long_df)

    long_df.columns.name = None

    return long_df

def add_ball_rows(df_wide: pd.DataFrame, players_long: pd.DataFrame) -> pd.DataFrame:
    """
    Replaces ball columns with its own designated row in the tracking data to avoid unnecessary duplicates
    """
    key_cols = ["period_id", "timestamp", "frame_id", "ball_state", "ball_owning_team_id"]

    ball_long = (
        df_wide[key_cols + ["ball_x", "ball_y", "ball_z", "ball_speed"]]
        .rename(columns={"ball_speed": "s"})
        .assign(
            object_id="BALL",
            x=lambda d: d["ball_x"],
            y=lambda d: d["ball_y"],
            z=lambda d: d["ball_z"],
        )
        .drop(columns=["ball_x", "ball_y", "ball_z"])
    )

    out = pd.concat([players_long, ball_long], ignore_index=True, sort=False)
    return out.drop(columns = ["ball_x", "ball_y", "ball_z", "ball_speed"])

In [42]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mplsoccer import Pitch

def animate_event(
    tracking,
    ball_id="BALL",
    frame_col="frame_id",      # or "timestamp"
    pitch_length=105,
    pitch_width=68,
):
    """
    Create an animation of a tracking event.

    tracking: pd.DataFrame with columns [frame_col, team, x, y]
              one row per entity per frame.
    save_path: if provided, saves animation to mp4/gif (based on extension).
    Returns: (fig, anim)
    """
    teams = tracking["team"].unique()
    team1_id, team2_id = teams[teams != "BALL"]
    # Unique frames in order
    frames = np.sort(tracking[frame_col].unique())

    # Pitch setup
    pitch = Pitch(
        pitch_type="custom",
        pitch_length=pitch_length,
        pitch_width=pitch_width
    )
    fig, ax = pitch.draw()

    # Create empty scatter artists once (then update their offsets each frame)
    scat_t1 = pitch.scatter([], [], ax=ax)
    scat_t2 = pitch.scatter([], [], ax=ax)
    scat_ball = pitch.scatter([], [], ax=ax)

    # Style them
    scat_t1.set_color("blue")
    scat_t2.set_color("red")
    scat_ball.set_color("black")

    # Optional: nicer sizing
    scat_t1.set_sizes([40])
    scat_t2.set_sizes([40])
    scat_ball.set_sizes([25])

    # If you want frame label on plot
    title = ax.set_title("")

    def to_pitch_xy(df):
        # Your original code shifts coords from centered to pitch coords
        x = df["x"].to_numpy() + pitch_length / 2
        y = df["y"].to_numpy() + pitch_width / 2
        return np.column_stack([x, y])

    def init():
        scat_t1.set_offsets(np.empty((0, 2)))
        scat_t2.set_offsets(np.empty((0, 2)))
        scat_ball.set_offsets(np.empty((0, 2)))
        title.set_text("")
        return scat_t1, scat_t2, scat_ball, title

    def update(i):
        fr = frames[i]
        f = tracking[tracking[frame_col] == fr]

        t1 = f[f["team"] == team1_id]
        t2 = f[f["team"] == team2_id]
        ball = f[f["team"] == ball_id]
        scat_t1.set_offsets(to_pitch_xy(t1) if len(t1) else np.empty((0, 2)))
        scat_t2.set_offsets(to_pitch_xy(t2) if len(t2) else np.empty((0, 2)))
        scat_ball.set_offsets(to_pitch_xy(ball) if len(ball) else np.empty((0, 2)))

        title.set_text(f"{frame_col}: {fr}")
        return scat_t1, scat_t2, scat_ball, title

    anim = FuncAnimation(
        fig,
        update,
        frames=len(frames),
        init_func=init,
        blit=True
    )


    return fig, anim




In [43]:
test = t[t['EVENT_ID'] == 18460600001695]
ooi = test[pd.isna(test['RECFRM'])][['FRAME_NUMBER', 'n_RECFRM', 'result', 'SUBTYPE']]
idx = 0
roi = ooi.iloc[idx]
start, end, result, subtype = [roi['FRAME_NUMBER'], roi['n_RECFRM'] + 10, roi['result'], roi['SUBTYPE']]
print(start, end, result, subtype)


event_t = tracking[tracking['frame_id'].isin(range(int(start), int(end)))]
c_frame = tracking_wide_to_long(event_t)
c_frame_ball = add_ball_rows(event_t, c_frame)
c_frame_ball['team'] = c_frame_ball['object_id'].map(player_map)
c_frame_ball = c_frame_ball[pd.notna(c_frame_ball['x'])]

fig, anim = animate_event(c_frame_ball, frame_col="frame_id")
from IPython.display import HTML
HTML(anim.to_jshtml())

IndexError: single positional indexer is out-of-bounds